# Temporal - Difference (Q) learning for game of Gridworld with VW

We are using vw for value estimate.. Seems to work really well when environment returns reward with every single move.
Model still struggles in some configurations (doesn't reach win after 15 moves), may be more epochs help?
Design matrix is simple sparse representation of state and objects with interactions with action.
Gridworld (4x4 grid, 4 objects - player, wall, win, pit) - possible states (when all objects can be initialized randomly) 24P4=500k)

Summary so far:
Trained Model wins about 30% of games
Vs Random model wins 1-2% of games

In [1]:
from games.gridworld.gridworld import GridWorld
from bandits import BanditAlgorithm
import pandas as pd
import rl_learning as rl
from IPython.display import display, HTML
import cPickle as pickle

In [2]:
def test_training_TD_for_gridworld(model_class, epochs, train=True):
    gridworld = GridWorld()
    if train:
        policy, model = rl.train_reinforcement_strategy_temporal_difference(epochs=epochs, game_obs=gridworld, model_class=model_class)

def test_policy(game_obs, model=None):
    print "---------- Testing policy:-----------"
    banditAlgorithm = BanditAlgorithm(params=0.1)
    game_obs.initiate_game()
    print "Initial state:"
    print game_obs.state
    game_obs.display_grid()
    move = 1

    # Unpickle if model obs not provided
    if not model:
        model = pickle.load(open(game_obs.base_folder_name + '/model_obs.pkl', mode='rb'))

    if model.model_class == 'vw_python':
        from vowpal_wabbit import pyvw
        model.model = pyvw.vw("--quiet -i {0}".format(model.model_path))
    while game_obs.game_status == 'in process':
        new_qval_table = banditAlgorithm.return_decision_reward_tuples(game_obs.state, model, game_obs.all_possible_decisions)
        best_action, value_estimate = banditAlgorithm.return_decision_with_max_reward(new_qval_table)
        print('Move #: %s; Taking action: %s' % (move, best_action))
        reward = game_obs.play(best_action)
        print game_obs.state
        game_obs.display_grid()
        if game_obs.game_status != 'in process': print "Summary: " + game_obs.game_status + " :Player Reward: " + str(reward)
        move += 1

        if move > 15:
            print "Too many moves"
            break

In [3]:
#test_training_TD_for_gridworld(model_class='vw_python', epochs=10000, train=True)

In [13]:
gridworld = GridWorld()
test_policy(gridworld)

---------- Testing policy:-----------
Initial state:
(coordinates(x=1, y=0), coordinates(x=2, y=3), coordinates(x=3, y=0), coordinates(x=0, y=3))
   0 1 2  3
0         +
1  P       
2         W
3  -       
Move #: 1; Taking action: right
(coordinates(x=1, y=1), coordinates(x=2, y=3), coordinates(x=3, y=0), coordinates(x=0, y=3))
   0  1 2  3
0          +
1     P     
2          W
3  -        
Move #: 2; Taking action: right
(coordinates(x=1, y=2), coordinates(x=2, y=3), coordinates(x=3, y=0), coordinates(x=0, y=3))
   0 1  2  3
0          +
1       P   
2          W
3  -        
Move #: 3; Taking action: up
(coordinates(x=0, y=2), coordinates(x=2, y=3), coordinates(x=3, y=0), coordinates(x=0, y=3))
   0 1  2  3
0       P  +
1           
2          W
3  -        
Move #: 4; Taking action: left
(coordinates(x=0, y=1), coordinates(x=2, y=3), coordinates(x=3, y=0), coordinates(x=0, y=3))
   0  1 2  3
0     P    +
1           
2          W
3  -        
Move #: 5; Taking action: down
(coordi